# Generación de Espectrogramas para Clasificación Musical

Este notebook implementa la generación de espectrogramas de mel a partir de archivos de audio musicales. Los espectrogramas generados serán utilizados posteriormente para entrenar una red neuronal en la tarea de clasificación de géneros musicales.

## Dependencias
- librosa: Procesamiento de audio
- matplotlib: Visualización
- numpy: Operaciones numéricas
- audiomentations: Aumentación de datos de audio

In [ ]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift

## 1. Generación de Espectrogramas Base

La siguiente función procesa archivos de audio y genera espectrogramas de mel segmentados. Cada archivo de audio se divide en segmentos de duración fija, y para cada segmento se genera un espectrograma.

### Parámetros:
- `input_path`: Ruta a la carpeta que contiene los archivos de audio
- `segment_duration`: Duración en segundos de cada segmento de audio

In [ ]:
def create_spectrograms(input_path, segment_duration):
    """Genera espectrogramas de mel a partir de archivos de audio.
    
    Args:
        input_path (str): Ruta a la carpeta con archivos de audio
        segment_duration (int): Duración en segundos de cada segmento
    """
    # Obtener el nombre del género musical desde la carpeta
    genre = os.path.basename(input_path)
    
    # Procesar cada archivo en la carpeta
    for file in os.listdir(input_path):
        audio_file = os.path.join(input_path, file)
        
        try:
            # Cargar el archivo de audio
            y, sr = librosa.load(audio_file)
            
            # Calcular la duración total
            duration = len(y)/sr
            start_sec = 0
            
            # Generar espectrogramas por segmentos
            for i in range(0, int(duration/segment_duration)):
                # Configurar el tamaño de la figura (216x224 píxeles)
                fig = plt.figure(figsize=(216/100, 224/100))
                ax = fig.add_subplot(1, 1, 1)
                fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
                
                # Extraer segmento y generar espectrograma
                segment = y[int(start_sec*sr):int((start_sec+segment_duration)*sr)]
                mel = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=224)
                mel_db = librosa.power_to_db(mel)
                
                # Visualizar y guardar el espectrograma
                librosa.display.specshow(mel_db, sr=sr, cmap='magma')
                
                # Preparar ruta de salida
                output_path = f'C:\Diego\music-genre-classifier\imagenes\gtzan2\{genre}'
                os.makedirs(output_path, exist_ok=True)
                
                # Guardar imagen
                plt.savefig(f'{output_path}/{file}_{i}.png')
                plt.clf()
                
                start_sec += segment_duration
                
        except Exception as e:
            print(f"Error procesando {audio_file}: {str(e)}")
            continue

## 2. Aumentación de Datos de Audio

Para mejorar la robustez del modelo, implementamos técnicas de aumentación de datos sobre los archivos de audio. Esto incrementa la diversidad del conjunto de datos y ayuda a prevenir el sobreajuste.

### Técnicas implementadas:
1. Adición de ruido gaussiano
2. Estiramiento temporal
3. Desplazamiento de tono

In [ ]:
def apply_audio_augmentations(folder_path, genre):
    """Aplica técnicas de aumentación a archivos de audio y genera espectrogramas.
    
    Args:
        folder_path (str): Ruta a la carpeta con archivos de audio
        genre (str): Nombre del género musical para organizar la salida
    """
    # Configurar pipeline de aumentación
    augment = Compose([
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
        TimeStretch(min_rate=0.8, max_rate=1.1, p=0.5),
        PitchShift(min_semitones=-4, max_semitones=4, p=0.5)
    ])
    
    for filename in os.listdir(folder_path):
        audio_path = os.path.join(folder_path, filename)
        
        try:
            # Cargar y aumentar el audio
            audio, sr = librosa.load(audio_path)
            augmented_audio = augment(samples=audio, sample_rate=sr)
            
            # Generar espectrogramas del audio aumentado
            output_path = f'C:/Diego/music-genre-classifier/imagenes/gtzan2/{genre}'
            create_spectrograms_from_audio(
                augmented_audio,
                sr,
                segment_duration=5,
                output_path=output_path,
                filename_prefix=f'{filename}_augmented'
            )
            
        except Exception as e:
            print(f"Error en la aumentación de {filename}: {str(e)}")

def create_spectrograms_from_audio(audio, sr, segment_duration, output_path, filename_prefix):
    """Genera espectrogramas a partir de un array de audio.
    
    Args:
        audio (np.array): Señal de audio
        sr (int): Tasa de muestreo
        segment_duration (int): Duración de cada segmento en segundos
        output_path (str): Ruta donde se guardarán los espectrogramas
        filename_prefix (str): Prefijo para los nombres de archivo
    """
    duration = len(audio) / sr
    start_sec = 0
    
    for i in range(0, int(duration / segment_duration)):
        # Configurar figura
        fig = plt.figure(figsize=(216/100, 224/100))
        fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
        
        # Generar espectrograma del segmento
        segment = audio[int(start_sec * sr):int((start_sec + segment_duration) * sr)]
        mel = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=224)
        mel_db = librosa.power_to_db(mel)
        
        # Visualizar y guardar
        librosa.display.specshow(mel_db, sr=sr, cmap='magma')
        os.makedirs(output_path, exist_ok=True)
        plt.savefig(f'{output_path}/{filename_prefix}_{i}.png')
        
        plt.close()
        start_sec += segment_duration

## 3. Procesamiento Principal

Ejecutamos el procesamiento sobre todo el conjunto de datos GTZAN, generando tanto espectrogramas base como aumentados.

In [ ]:
# Configuración de rutas
BASE_PATH = "C:/Diego/music-genre-classifier/audio/gtzan"

# Obtener lista de géneros (directorios)
genres = [d for d in os.listdir(BASE_PATH) if os.path.isdir(os.path.join(BASE_PATH, d))]

# Procesar cada género
for genre in genres:
    genre_path = os.path.join(BASE_PATH, genre)
    print(f"Procesando género: {genre}")
    
    # Generar espectrogramas base
    create_spectrograms(genre_path, segment_duration=5)
    
    # Aplicar aumentación y generar espectrogramas adicionales
    apply_audio_augmentations(genre_path, genre)

print("Procesamiento completado.")